In [1]:
# Stock Recommendations Aggregator
# Author: Balie
# Email: baliepages@gmail.com

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.max_columns", 100)
pd.options.mode.chained_assignment = None 

iDf = pd.read_excel(r'StocksOutput\WatchList.xlsx', sheet_name='Stocks')
stockArray = iDf['Ticker'].tolist()

In [3]:
import datetime as dt

#Cosmetic function to write comment to log
def drawLine(comment):
    try:
        print('', end = '\n')
        print(comment + ' ' * 10 + '-' * 10 + str(dt.datetime.now().strftime("%H:%M:%S")), end = '')
        print('\n')
    except Exception as e:
        errMsg = 'Error: Draw line'
        print(errMsg, repr(e))


In [4]:
import tldextract

loadStartTime = dt.datetime.now()
loadID = int(''.join(x for x in str(loadStartTime) if x.isdigit())[:14])

global urlArray, stockArray, siteArray, stockDf
urlArray = []
stockDf = pd.DataFrame(columns = ["siteName", "url", "stockCode", "snipCode", "snipText"])

global greedy, lazyloaders
greedy = ['marketbeat', 'zacks']
lazyLoaders = ['investorsobserver', 'weissratings', 'gurufocus', 'marketbeat', 'zacks']

global selectorDict
selectorDict = {
'wsj'           : 'snip = soup.select(".at8-col4 > .zonedModule")[0]',
'marketwatch'   : 'snip = soup.find(True, {"class" : "analyst-ratings"})',
'thestreet'     : 'snip = soup.select("#promo-div-quote-nav-unit-small-screen > div.quote-nav-rating-qr-label-container")[0]',
#'zacks'        : 'snip = soup.find(True, {"class" : ["rank_view"]})',
#'zacks2'       : 'snip = soup.select(#premium_research > div > dl:nth-child(4) > dt > p")',
'zacks'         : ".rank_view",
'zacks2'        : "#premium_research > div > dl:nth-child(4) > dt > p",
'investorsobserver' : ".bar-label",
'weissratings'  : ".r-rating",
'gurufocus'     : "#def_body_detail_height > font:nth-child(2)",
'marketbeat'    : "#cphPrimaryContent_cphTabContent_pnlChart > div.row > div:nth-child(1) > div > div > div.col-sm-auto",
'marketbeat2'   : "#cphPrimaryContent_cphTabContent_pnlChart > div.row > div:nth-child(2) > div > div > div:nth-child(1) > h3"
}

#Source websites
dummyStock = 'pangu'
wsjUrl     = 'https://www.wsj.com/market-data/quotes/' + dummyStock + '/research-ratings' 
mWatchUrl  = 'https://www.marketwatch.com/investing/stock/' + dummyStock + '/analystestimates'
strUrl     = 'https://realmoney.thestreet.com/quote/' + dummyStock + '.html'
zacUrl     = 'https://www.zacks.com/stock/quote/' + dummyStock 
invoUrl    = 'https://www.investorsobserver.com/symbols/' + dummyStock
wrUrl      = 'https://weissratings.com/stock/' + dummyStock + '-nasdaq'
gurUrl     = 'https://www.gurufocus.com/term/fscore/' + dummyStock + '/Piotroski-F-Score'
mBeatUrl   = 'https://www.marketbeat.com/stocks/NASDAQ/' + dummyStock + '/price-target'

siteArray  = [wsjUrl, mWatchUrl, strUrl, zacUrl, invoUrl, wrUrl, gurUrl, mBeatUrl]
stockArray  = ['UBER','C', 'PLUG', 'CMCSA', 'GOLD', 'JPM', 'MTTR', 'GPN', 'SOFI']
#urlArray.remove(['TWLO', 'https://www.wsj.com/market-data/quotes/TWLO/financials'])

stockArray  = sorted(stockArray)
siteArray  = sorted(siteArray)

#Build Site Url Array
for st in range(len(stockArray)):
    stockCode = stockArray[st]
    for si in range(len(siteArray)):
        site = siteArray[si]
        site = site.replace(dummyStock, stockCode)
        urlArray.append([stockCode, site])

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
from cryptography.utils import CryptographyDeprecationWarning
warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)


import time
options = Options()
options.add_argument("--headless")
s=Service(r'C:\chromedriver.exe')
driver = webdriver.Chrome(service=s, options=options)
    
def seleniumScrape(stockCode, domain, url, selector):
    try:
        try:
            driver.get(url)
            time.sleep(1)
            snip_html = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('innerHTML')
            snip_text = driver.find_element(By.CSS_SELECTOR, selector).text
        except Exception as e:
            newUrl = url.replace('NASDAQ','NYSE').replace('nasdaq','nyse')
            print('Retry with NYSE', newUrl)
            driver.get(newUrl)
            time.sleep(1)
            snip_html = driver.find_element(By.CSS_SELECTOR, selector).get_attribute('innerHTML')
            snip_text = driver.find_element(By.CSS_SELECTOR, selector).text
            
        if (domain in greedy):
            selector2 = selectorDict.get((domain + str(2)))
            snip2_html = driver.find_element(By.CSS_SELECTOR, selector2).get_attribute('innerHTML')
            snip2_text = driver.find_element(By.CSS_SELECTOR, selector2).text
            snip_html = snip_html + 'serppu' + snip2_html
            snip_text = snip_text + 'serppu' + snip2_text
        
        newRow = {'siteName':domain, 'url':url, 'stockCode':stockCode, 'snipCode':str(snip_html), 'snipText': snip_text}
        return newRow

    except Exception as e:
        print('Selenium Error')
        #print('Selenium Error: ', repr(e))

In [6]:
from bs4 import BeautifulSoup
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

scrapeStartTime = dt.datetime.now()
drawLine('Scraping Started... Grab your coffee! ')

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

urlCount = len(urlArray)
for i in range(len(urlArray)):
    try:
        stockCode = urlArray[i][0]
        url = urlArray[i][1]
        parsedUrl = tldextract.extract(url)
        domain = parsedUrl.domain
        selector = selectorDict.get(domain)
        print(str(i) + '/' + str(urlCount) + ' : ', stockCode + ' : ', domain + ' : ', url)

        #Using Selenium for lazy loading sites
        if (domain in lazyLoaders):
            newRow = seleniumScrape(stockCode, domain, url, selector)
            stockDf = stockDf.append(newRow, ignore_index=True)
        else:
            try:
                r = requests.get(url, headers=headers, allow_redirects=False, verify=False)
                soup = BeautifulSoup(r.text)
                exec(selector)
            except Exception as e:
                newUrl = url.replace('NASDAQ','NYSE').replace('nasdaq','nyse')
                print('Retry with NYSE', newUrl)
                r = requests.get(newUrl, headers=headers, allow_redirects=False, verify=False)
                soup = BeautifulSoup(r.text)
                exec(selector)
                
            snip_text = snip.get_text().strip()
            snip = str(snip)

            if (domain in greedy):
                selector = selectorDict.get((domain + str(2)))
                exec(selector)
                snip2_text = snip2.get_text().strip()
                snip2 = str(snip2)
                snip = snip + 'serppu' + snip2
                snip_text = snip_text + 'serppu' + snip2_text

            newRow = {'siteName':domain, 'url':url, 'stockCode':stockCode, 'snipCode':snip, 'snipText': snip_text}
            stockDf = stockDf.append(newRow, ignore_index=True)

    except Exception as e:
        print('BS4 Error')
        #print('BS4 Error: ', repr(e))
        
scrapeEndTime = dt.datetime.now()
loadDuration = str(scrapeEndTime - scrapeStartTime).split('.')[0]
drawLine('Scraping Completed in ' + loadDuration)


Scraping Started... Grab your coffee!           ----------09:17:01

0/72 :  C :  thestreet :  https://realmoney.thestreet.com/quote/C.html
1/72 :  C :  weissratings :  https://weissratings.com/stock/C-nasdaq
Retry with NYSE https://weissratings.com/stock/C-nyse
2/72 :  C :  gurufocus :  https://www.gurufocus.com/term/fscore/C/Piotroski-F-Score
3/72 :  C :  investorsobserver :  https://www.investorsobserver.com/symbols/C
4/72 :  C :  marketbeat :  https://www.marketbeat.com/stocks/NASDAQ/C/price-target
Retry with NYSE https://www.marketbeat.com/stocks/NYSE/C/price-target
5/72 :  C :  marketwatch :  https://www.marketwatch.com/investing/stock/C/analystestimates
6/72 :  C :  wsj :  https://www.wsj.com/market-data/quotes/C/research-ratings
7/72 :  C :  zacks :  https://www.zacks.com/stock/quote/C
Selenium Error
8/72 :  CMCSA :  thestreet :  https://realmoney.thestreet.com/quote/CMCSA.html
9/72 :  CMCSA :  weissratings :  https://weissratings.com/stock/CMCSA-nasdaq
10/72 :  CMCSA :  gurufo

In [7]:
df = stockDf.copy()

In [8]:
import json

def renderHtml(snip):
    try:
        return '{0}.prettify.stripped_strings'.format(snip)
    except Exception as e:
        pass

def getTableData(snip):
    try:
        return [[cell.text for cell in row("td")] for row in BeautifulSoup(snip)("tr")]
    except Exception as e:
        pass
    
def linearizeTable(snip):
    try:
        return [list(map(lambda x: str(x).replace('\n',''), item)) for item in snip if item]
    except Exception as e:
        pass
    
def tableDataToDict(data):
    try:
        cols = ['Rating', '3M', '1M', 'Current']
        df = pd.DataFrame(data, columns=cols)
        return df.set_index('Rating').T.to_dict('list')
    except Exception as e:
        pass
    
def listToDict(lister):
    try:
        return dict(lister)
    except Exception as e:
        pass

df['snipCode_v1'] = df['snipCode'].apply(getTableData)
df['snipCode_v2'] = df['snipCode_v1'].apply(linearizeTable)
df['snipCode_v3'] = df['snipCode_v2'].apply(tableDataToDict)
df['snipCode_v2'] = df['snipCode_v2'].apply(listToDict)
#df.style.format({'SnipCode': renderHtml})


In [9]:
#Recommendation Matrix
df['snipText'] = df['snipText'].apply(lambda x: x.replace('\n', '')
                                                 .replace('TheStreetQuant Rating:', '')
                                                 .replace('(As of Today)', '')
                                                 .replace(':', '')
                                                 .replace('Based on','')
                                                 .replace('Analyst Ratings','')
                                                 .replace('serppu','')
                                     )
recDf = (df.query('siteName in ["zacks", "thestreet", "investorsobserver", "weissratings", "gurufocus", "marketbeat"]')
          #.query('snipCode.str.contains("Buy")')
        [['stockCode', 'siteName', 'url', 'snipText']]
)
recDfLinear = pd.pivot_table(recDf, values = 'snipText', index=['stockCode'], columns = 'siteName', aggfunc='first').reset_index()

In [10]:
import json
json_struct = json.loads(df.to_json(orient="records"))    
dfFlat = pd.json_normalize(json_struct)
dfFlat.rename(columns=lambda x: x.replace('snipCode_v3.','').replace('snipCode_v2.',''), inplace=True)
dfTrend = dfFlat.query('siteName == "marketwatch"')[['stockCode', 'Buy', 'Sell', 'Consensus']]
dfPrice = dfFlat.query('siteName == "wsj"')[['stockCode', 'Average', 'Current Price']]

In [11]:
#Merge all dataframes
df1 = recDfLinear.merge(dfTrend, how = 'outer', on='stockCode')
dfMerge = df1.merge(dfPrice, how = 'outer', on='stockCode')
dfCols = ['stockCode', 'Current Price', 'Average', 
          'gurufocus', 'investorsobserver', 'thestreet', 'weissratings', 
          'zacks', 'marketbeat',
         'Buy', 'Sell', 'Consensus']
dfMerge = dfMerge[dfCols]
dfMerge.sort_values(by=['investorsobserver'], inplace=True, ascending=False)

In [12]:
def futureWork():
    
    #Sell Code to Number
    ratingDict = {
        'Strong Buy' : 3,
        'Buy' : 2,
        'Overweight' : 1,
        'Hold' : 0,
        'Underweight' : -1,
        'Sell' : -2,
        'Strong Sell' : -3,
        'A' : 3,
        'B' : 2,
        'C' : 1,
        'D' : 0,
        'E' : -1,
        'F' : -2,
        'A+ Buy' : 3,
        'A Buy' : 2.8,
        'A- Buy' : 2.6,
        'B+ Buy' : 2.4,
        'B Buy' : 2.2,
        'B- Buy' : 2,
        'C+ Hold' : 1,
        'C Hold' : 0,
        'C- Hold' : -1,
        'D+ Sell' : -2,
        'D Sell' : -2.2,
        'D- Sell' : -2.4,
        'E+ Sell' : -2.6,
        'E Sell' : -2.8,
        'E- Sell' : -3
    }
    
    for x in CodeToNumCols:
        df.replace({x: ratingDict}, inplace = True)
        pass

    #To Numeric
    df['mb_Rating'] = pd.to_numeric(df['mb_Rating'], errors='coerce') *20

    df['zacksScore'] = ''
    df['zacksVGM'] = ''
    try:
        df['zacksScore'] = df[df.zacks.str.contains('of 5',case=False)]['zacks'].apply(lambda x: x.split(']')[1].split('5 ')[1])
        df['zacksVGM'] = df[df.zacks.str.contains('VGM',case=False)]['zacks'].apply(lambda x: x.split(']')[0].split('|')[3].replace(' VGM', '').strip())
    except Exception as e:
        drawLine('ERROR : ' + repr(e))
    
    #Average Target Price
    TgtPriceCols = ['invo_Price_Tgt', 'wsj_Tgt', 'mb_TgtPrice']
    df[TgtPriceCols] = df[TgtPriceCols].replace(0, np.nan)
    df['TgtPrice_Avg'] = df[TgtPriceCols].mean(axis=1)
    df['Growth_Avg'] = ((df['TgtPrice_Avg'] - df['wsj_Current']) / df['wsj_Current'])*100
    
    #Existence of Buy or Sell
    df['StrongSellSignal'] = (df[recCols] < -1 ).any(axis=1)
    df['StrongBuySignal'] = (df[recCols] > 2.2 ).any(axis=1)
    
    #Moving Average
    df['MA'] = (df['200MA'] + df ['50MA'])/2
    df['MA_To_Curr'] =  (df ['MA'] - df['wsj_Current']) / df['wsj_Current']

    #Downgraded Flag
    df['Downgraded'] = (df['mw_Mean'] < df['mw_Mean1']) | (df['mb_Buy'] < df['mb_Buy1'])
    df['Upgraded'] = (df['mw_Mean'] > df['mw_Mean1']) | (df['mb_Buy'] > df['mb_Buy1'])

In [13]:
#List of all Dataframes to be written to the file
dflist = [dfMerge]

tgtFileName = "StocksOutput/StockMarket_" + str(loadID) + ".xlsx"
Excelwriter = pd.ExcelWriter(tgtFileName, engine="xlsxwriter")
for i, df in enumerate (dflist):
    df.to_excel(Excelwriter, sheet_name="Sheet" + str(i+1), index=False)
Excelwriter.save()

loadEndTime = dt.datetime.now()
loadDuration = str(loadEndTime - loadStartTime).split('.')[0]
drawLine('Load Completed in ' + loadDuration)


Load Completed in 0:03:28          ----------09:20:25

